#### Goal for this webscrape phase: "go into" a page with links to "zoom in"/loop through links that then take you to another level that will then loop through links to scrape each webpage with in
    Challenge:
    -there are 14 styles (categories) and around ~120 links (subcategories)
    -each style can have multiptle pages to loop through

###### Starting small by scraping one page

In [115]:
import cloudscraper
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd  
import time 
from random import randint


In [ ]:
url = "https://www.beeradvocate.com/beer/profile/143/1352/" 
# url ="https://www.beeradvocate.com/beer/profile/17033/53930/"
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
scraper = cloudscraper.create_scraper() 
info = scraper.get(url) 
print(info.status_code) 
 

In [ ]:
soup = BeautifulSoup(info.text, 'lxml') 

In [ ]:
name=soup.h1.find(string=True)
name

In [ ]:
soup.find('h1').text
#will give me both the beer title and brewery merged together

In [ ]:
# soup.find('dl',{'class':'beerstats'}).text

In [ ]:
brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
brewery

In [ ]:
loaction=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
loaction

In [ ]:
location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
location2

In [ ]:
style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
style

In [ ]:
ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
ba_score
##official score on ba site 74::'okay'

In [ ]:
over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
over_all_rank
##over all rank is against all beers in ba site

In [ ]:
style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
style_rank
## style rank is among all beers within style category

In [ ]:
alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
alcohol_percent

In [ ]:
avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
avg_rating
# Average across all ratings for this beer

In [ ]:
percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
percent_dev
# Percentage of deviation within the ratings

In [ ]:
review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
review_count
# Number of reviews for this beer

In [ ]:
rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
rating_count
# Number of ratings for this beer

In [ ]:
status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
status
# the beer's current activity status

In [ ]:
date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
date_added
# The date this beer was added to the site

###### zooming out one level to scrape through "bocks" pg 1 with 50 results

In [ ]:
# youtube page: John Watson Rooney helped get more of the wabscraping process 
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
website=f'{root}/beer/styles/32/?sort=revsD&start=0'
scraper = cloudscraper.create_scraper() 
info = scraper.get(website) 
print(info.status_code)


In [ ]:
soup = BeautifulSoup(info.text, 'lxml')

In [ ]:
table=soup.find('table')
# table is where the links are concentrated in 

In [ ]:
table.find_all("a", href=re.compile("profile"))[::2]

In [ ]:
links=[link['href'] for link in table.find_all("a", href=re.compile("profile"))[::2]]
print(links)

In [ ]:
beerlinks=[]
for link in table.find_all("a", href=re.compile("profile"))[::2]:
    beerlinks.append(root + link['href'])


In [ ]:
beerlinks


##### Building together code to loop through links on one page  

In [ ]:
# print(f'{variable im using in the for loop} : {variable 1 i am getting out of the loop this pass}, {variable 2 im getting out of the loop this pass}')
# https://medium.com/swlh/improve-your-web-scraper-with-limited-retry-loops-python-35e21730cbf5
# https://www.youtube.com/watch?v=sIOMDu6MXJQ&list=PLRzwgpycm-Fio7EyivRKOBN4D3tfQ_rpu&index=3
# https://www.youtube.com/watch?v=8dTpNajxaH0&t=503s
# https://www.freecodecamp.org/news/how-to-scrape-multiple-web-pages-using-python/

In [ ]:
# testlink='https://www.beeradvocate.com/beer/profile/143/1352/'
# for page pagination, ran a range to an imposiblly high number, by 50  (results per page that url uses to fill start param)
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
website=f'{root}/beer/styles/32/?sort=revsD&start=100'
scraper = cloudscraper.create_scraper() 
response = scraper.get(website)  
soup = BeautifulSoup(response.text, 'lxml')
table=soup.find('table')
# print(table)
beerlist=[]

beerlinks=[]
for link in table.find_all("a", href=re.compile("profile"))[::2]:
    beerlinks.append(root + link['href'])
# print(beerlinks)

for link in beerlinks:
    scraper = cloudscraper.create_scraper()
    r=scraper.get(link, headers=header)
    soup=BeautifulSoup(r.text,'lxml')
    name=soup.h1.find(string=True)
    try:    
        brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
    except:
        brewery= 'Na'
    try:
        location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
    except:
        location= 'Na'
    try:
        location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
    except:
        location2= 'Na'
    try:
        style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
    except:
        style= 'Na'
    try:
        ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
    except:
        ba_score= 'Na'
    try:
        over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
    except:
        over_all_rank= 'Na'
    try:
        style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
    except:
        style_rank= 'Na'
    try:
        alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
    except:
        alcohol_percent= 'Na'
    try:
        avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
    except:
        avg_rating= 'Na'
    try:
        percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
    except:
        percent_dev= 'Na'
    try:
        review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
    except:
        review_count= 'Na'
    try:
        rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
    except:
        rating_count= 'Na'
    try:
        status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
    except:
        status= 'Na'
    try:
        date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
    except:
        date_added= 'Na'
    beer_info= {
                'name': name,
                'brewery': brewery,
                'location':location,
                'location2':location2,
                'style': style,
                'ba_score':ba_score,
                'over_all_rank': over_all_rank,
                'style_rank': style_rank,
                'alcohol_percent': alcohol_percent,
                'avg_rating':avg_rating,
                'percent_dev': percent_dev,
                'review_count': review_count,
                'rating_count':rating_count,
                'status': status,
                'date_added': date_added
                }
    beerlist.append(beer_info)
    print(f'{link}-{name}')

# print({variable 1 i am getting out of the loop this pass}, {variable 2 im getting out of the loop this pass}')

In [ ]:
print(beerlist)

In [ ]:
df=pd.DataFrame(beerlist)
df

In [ ]:
len(beerlinks)

##### Building together code to loop through links of different styles of beer (zooming out another level/ another loop)
-challanges: url for different beer styles has a slight change in on of the parameters for each style of beer, need to be mindful of this in loop, meaing a few changes must be implimented on code above


In [ ]:
# root='https://www.beeradvocate.com'
# header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 

    
# website=f'{root}/beer/styles/'
# scraper = cloudscraper.create_scraper() 
# info = scraper.get(website)  
# soup = BeautifulSoup(info.text, 'lxml')
# container=soup.find('div',{'id':'ba-content'})
# container.find_all('a', href=True)[0:-1]

# style_links=[]
# for link in container.find_all('a', href=True)[0:-1]:
#     style_links.append(link['href'])

#  beerlinks=[]
#         for link in table.find_all("a", href=re.compile("profile"))[::2]:
#             beerlinks.append(root + link['href'])

In [ ]:
# style_links

In [ ]:
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 

website=f'{root}/beer/styles/'
scraper = cloudscraper.create_scraper() 
info = scraper.get(website)  
soup = BeautifulSoup(info.text, 'lxml')
container=soup.find('div',{'id':'ba-content'})

beerlist=[]

style_href=[]
for link in container.find_all('a', href=True)[0:-1]:
    style_href.append(link['href'])

for h in style_href:
    webber=f'{root}{h}'
    time.sleep(5)
    print(webber)
    

#     for x in range(0,100000,50):
#         print(requests.get(f'{root}/beer/styles/32/?sort=revsD&start={x}'))
#         if requests.get(f'{root}/beer/styles/32/?sort=revsD&start={x}').status_code == 200:    
#             web=f'{webber}?sort=revsD&start={x}'
#             scraper = cloudscraper.create_scraper() 
#             info = scraper.get(web)  
#             soup = BeautifulSoup(info.text, 'lxml')
#             table=soup.find('table')

#             beerlinks=[]
#             for link in table.find_all("a", href=re.compile("profile"))[::2]:
#                 beerlinks.append(root + link['href'])


#             for link in beerlinks:
#                 r=requests.get(link, headers=header)
#                 soup=BeautifulSoup(r.content,'lxml')
#                 name=soup.h1.find(string=True)
#                 try:    
#                     brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
#                 except:
#                     brewery= 'Na'
#                 try:
#                     location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
#                 except:
#                     location= 'Na'
#                 try:
#                     location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
#                 except:
#                     location2= 'Na'
#                 try:
#                     style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
#                 except:
#                     style= 'Na'
#                 try:
#                     ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
#                 except:
#                     ba_score= 'Na'
#                 try:
#                     over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
#                 except:
#                     over_all_rank= 'Na'
#                 try:
#                     style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
#                 except:
#                     style_rank= 'Na'
#                 try:
#                     alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
#                 except:
#                     alcohol_percent= 'Na'
#                 try:
#                     avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
#                 except:
#                     avg_rating= 'Na'
#                 try:
#                     percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
#                 except:
#                     percent_dev= 'Na'
#                 try:
#                     review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
#                 except:
#                     review_count= 'Na'
#                 try:
#                     rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
#                 except:
#                     rating_count= 'Na'
#                 try:
#                     status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
#                 except:
#                     status= 'Na'
#                 try:
#                     date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
#                 except:
#                     date_added= 'Na'
#                 beer_info= {
#                              'name': name,
#                              'brewery': brewery,
#                             'location':location,
#                             'location2':location2,
#                             'style': style,
#                             'ba_score':ba_score,
#                             'over_all_rank': over_all_rank,
#                             'style_rank': style_rank,
#                             'alcohol_percent': alcohol_percent,
#                             'avg_rating':avg_rating,
#                             'percent_dev': percent_dev,
#                             'review_count': review_count,
#                             'rating_count':rating_count,
#                             'status': status,
#                             'date_added': date_added
#                             }
#                 beerlist.append(beer_info)
#                 print(f'{h}-{x}-{link}-{name}')
#         else:
#             break          



In [ ]:
# root='https://www.beeradvocate.com'  #base url
# headers= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
# website=f'{root}/beer/styles/'  #base url + path to beer style list 
# scraper = cloudscraper.create_scraper() #calling a scraper session
# response = scraper.get(website,headers=headers)  #built scraper (documnetation says treat it like response)
# soup = BeautifulSoup(response.text, 'html.parser')  #soup of beer style list 
# container=soup.find('div',{'id':'ba-content'}) #getting section of html that contains most of href style ids'
# # print(response.status_code)    ##ran and got 200 scraper works

# beerlist=[]

style_href=[link['href'] for link in  container.find_all('a', href=True)][0:-1] #getting text of href that needs to later be paired with root and other parameters
# print(len(style_href)) ## 120 links from beer style list
for h in style_href: #contatinating root and h to pull up style lists
#     webber=f'{root}{h}'
#     print(f'{webber}')
for x in range(0,100000,50):
    m=f'{root}{h}?sort=revsD&start={x}'
    print(m)
#     scraper = cloudscraper.create_scraper()
    res = scraper.get(webber,headers=headers)
    time.sleep(3)
    print(scraper.get(f'{res}').status_code)
    print(response.status_code) ##ran 200 for each response from webber
    if scraper.get(webber).status_code == 200:   
        scraper = cloudscraper.create_scraper()
        response = scraper.get(webber,headers=headers)
#         time.sleep(3)
#         print(response.status_code)
        print(f'{webber}')
        for x in range(0,100000,50):
            web=f'{webber}{x}'
            if scraper.get(f'{webber}{x}').status_code == 200:        
                scraper = cloudscraper.create_scraper() 
                info = scraper.get(website)  
                soup = BeautifulSoup(info.text, 'lxml')
                table=soup.find('table')
                beerlinks=[root + link['href'] for link in table.find_all("a", href=re.compile("profile"))][::2]
                for link in beerlinks:
                    scraper = cloudscraper.create_scraper()
                    r=scraper.get(link, headers=header)
                    soup=BeautifulSoup(r.text,'lxml')
                    name=soup.h1.find(string=True)
                    try:    
                        brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
                    except:
                        brewery= 'Na'
                    try:
                        location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
                    except:
                        location= 'Na'
                    try:
                        location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
                    except:
                        location2= 'Na'
                    try:
                        style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
                    except:
                        style= 'Na'
                    try:
                        ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
                    except:
                        ba_score= 'Na'
                    try:
                        over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
                    except:
                        over_all_rank= 'Na'
                    try:
                        style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
                    except:
                        style_rank= 'Na'
                    try:
                        alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
                    except:
                        alcohol_percent= 'Na'
                    try:
                        avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
                    except:
                        avg_rating= 'Na'
                    try:
                        percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
                    except:
                        percent_dev= 'Na'
                    try:
                        review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
                    except:
                        review_count= 'Na'
                    try:
                        rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
                    except:
                        rating_count= 'Na'
                    try:
                        status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
                    except:
                        status= 'Na'
                    try:
                        date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
                    except:
                        date_added= 'Na'
                    beer_info= {
                                'name': name,
                                 'brewery': brewery,
                                'location':location,
                                'location2':location2,
                                'style': style,
                                'ba_score':ba_score,
                                'over_all_rank': over_all_rank,
                                'style_rank': style_rank,
                                'alcohol_percent': alcohol_percent,
                                'avg_rating':avg_rating,
                                'percent_dev': percent_dev,
                                'review_count': review_count,
                                'rating_count':rating_count,
                                'status': status,
                                'date_added': date_added
                                }
                    beerlist.append(beer_info)
                    print(f'{h}-{x}-{link}-{name}')
            else:
                break 


In [ ]:
def cook_soup(url):
    scraper = cloudscraper.create_scraper()
    headers= {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}
    response = scraper.get(url,headers=headers)
    soup = BeautifulSoup(response.text,'lxml')
    print(f'url: {url} - status:{response.status_code} -')
    

#will create an instance of cloudscraper session for url and print status code
# https://stackoverflow.com/questions/73027049/how-to-ensure-response-200-for-multiple-requests-when-webscraping

In [ ]:
test=cook_soup('https://www.beeradvocate.com/beer/profile/57/7714/')


In [ ]:
url="https://www.beeradvocate.com/beer/profile/417/47795/"
test_2=cook_soup(url)

In [ ]:
root='https://www.beeradvocate.com' # base url
website=f'{root}/beer/styles/' #url that has links to styles of beers (14 styles, and 120 links)
cook_soup(website) #makes soup of previous url
container=soup.find('div',{'id':'ba-content'}) #looks for place where href are
style_href=[link['href'] for link in  container.find_all('a', href=True)][0:-1] #saves hrefs
for h in style_href:
    webber=f'{root}{h}'
#     print(webber)
    cook_soup(webber)
    for x in range(0,100000,50):
        web=f'{root}{h}?sort=revsD&start={x}'
        cook_soup(web)
#         if response.status_code != 200:
#             break ## issue is that it keeps going after last results page with existing page with empty results
#         else:
        table=soup.find('table')
        beerlinks=[root + link['href'] for link in table.find_all("a", href=re.compile("profile"))][::2]
#         print(beerlinks)
# #         while beerlinks 
            
        
        
        
        
        
#             table=soup.find('table')
#             beerlinks=[root + link['href'] for link in table.find_all("a", href=re.compile("profile"))][::2]
#             for link in beerlinks:
#                 scraper = cloudscraper.create_scraper()
#                 r=scraper.get(link, headers=header)
#                 soup=BeautifulSoup(r.text,'lxml')
#                 name=soup.h1.find(string=True)
#                 try:    
#                     brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
#                 except:
#                     brewery= 'Na'
#                 try:
#                     location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
#                 except:
#                     location= 'Na'
#                 try:
#                         location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
#                 except:
#                         location2= 'Na'
#                 try:
#                         style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
#                 except:
#                         style= 'Na'
#                 try:
#                         ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
#                 except:
#                         ba_score= 'Na'
#                 try:
#                         over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
#                 except:
#                         over_all_rank= 'Na'
#                 try:
#                         style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
#                 except:
#                         style_rank= 'Na'
#                 try:
#                         alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
#                 except:
#                         alcohol_percent= 'Na'
#                 try:
#                         avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
#                 except:
#                         avg_rating= 'Na'
#                 try:
#                         percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
#                 except:
#                         percent_dev= 'Na'
#                 try:
#                         review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
#                 except:
#                         review_count= 'Na'
#                 try:
#                         rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
#                 except:
#                         rating_count= 'Na'
#                 try:
#                         status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
#                 except:
#                     status= 'Na'
#                 try:
#                     date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
#                 except:
#                     date_added= 'Na'
#                 beer_info= {
#                             'name': name,
#                             'brewery': brewery,
#                             'location':location,
#                             'location2':location2,
#                             'style': style,
#                             'ba_score':ba_score,
#                             'over_all_rank': over_all_rank,
#                             'style_rank': style_rank,
#                             'alcohol_percent': alcohol_percent,
#                             'avg_rating':avg_rating,
#                             'percent_dev': percent_dev,
#                             'review_count': review_count,
#                             'rating_count':rating_count,
#                             'status': status,
#                             'date_added': date_added
#                             }
#                 beerlist.append(beer_info)
#                 print(f'{h}-{x}-{link}-{name}')    
    
        